In [1]:
import numpy as np
import pickle
import json
import copy
import pandas as pd
from tqdm import tqdm
import random

from perturbations import *
from helpers import *

# Read in data

Vocab

In [2]:
from Vocab import Vocab
vocab = Vocab('data/vocab.csv', 3)

Initialising vocab from file.


Segment data

In [3]:
from SegmentDataset import SegmentDataset

segment_test_set = SegmentDataset(
    data_dir='data/',
    segment_file='segments.json',
    vectors_file='vectors.json',
    split='test'
)

print("Testing the SegmentDataset class item getter...")
print("Dataset contains {} segment samples".format(len(segment_test_set)))
sample_id = 2
sample = segment_test_set[sample_id]
print("Segment {}:".format(sample_id))
print("Image set: {}".format(sample["image_set"]))
print("Target image index(es): {}".format(sample["targets"]))
# print("Target image Features: {}".format([segment_test_set.image_features[sample["image_set"][int(target)]] for target in sample["targets"]]))
print("Encoded segment: {}".format(sample["segment"]))
print("Decoded segment dialogue: {}".format(vocab.decode(sample["segment"])))
print("Segment length: ", sample["length"])
print("\nDone.")

Testing the SegmentDataset class item getter...
Dataset contains 6801 segment samples
Segment 2:
Image set: ['93469', '380128', '14238', '259745', '524866', '341060', '483794', '96757']
Target image index(es): [6, 2, 3]
Encoded segment: [4, 21, 11, 757, 18, 6, 26, 49, 10, 5, 41]
Decoded segment dialogue: ['-A-', 'one', 'with', 'roses', 'and', 'a', 'white', 'laptop', '?', '-B-', 'nope']
Segment length:  11

Done.


Chain data

In [4]:
from ChainDataset import ChainDataset

print("Testing the ChainDataset class initialization...")

chain_test_set = ChainDataset(
    data_dir='data/',
    segment_file='segments.json',
    chain_file='test_chains.json',
    vectors_file='vectors.json',
    split='test'
)

print("Dataset contains {} cains.".format(len(chain_test_set.chains)))

sample_id = 5
sample = chain_test_set.chains[sample_id]

print("Chain {}:".format(sample_id))
print("Source Game ID: {}".format(sample["game_id"]))
print("Target image index: {}".format(sample["target"]))
print("Chain length: {}".format(len(sample["segments"])))
print("Segment IDs: {}".format(sample["segments"]))
print("Segment lengths: ", sample["lengths"])
print("First segment encoding: {}".format(chain_test_set.segments[sample["segments"][0]]["segment"]))
print("First segment decoded dialogue: {}".format(vocab.decode(chain_test_set.segments[sample["segments"][0]]["segment"])))
print("Second segment decoded dialogue: {}".format(vocab.decode(chain_test_set.segments[sample["segments"][1]]["segment"])))
# print("third segment decoded dialogue: {}".format(vocab.decode(chain_test_set.segments[sample["segments"][2]]["segment"])))

print("Reference chain and segments' associated image sets:")
for segment in sample["segments"]:
    print(vocab.decode(chain_test_set.segments[segment]["segment"]))
    print(chain_test_set.segments[segment]["image_set"])
print("\nDone.")

print("Testing the ChainDataset class item getter...")


Testing the ChainDataset class initialization...
Dataset contains 2811 cains.
Chain 5:
Source Game ID: 3R2UR8A0IBELL07JV1VG7ISBNQAOX23WMINLGALC18GY2NUMUBSANWXRHCAO
Target image index: 522129
Chain length: 4
Segment IDs: [3, 10, 12, 15]
Segment lengths:  [23, 23, 21, 19]
First segment encoding: [5, 38, 191, 23, 127, 11, 6, 120, 23, 127, 15, 9, 314, 10, 4, 13, 5, 80, 14, 128, 68, 22, 10]
First segment decoded dialogue: ['-B-', 'two', 'plates', 'of', 'food', 'with', 'a', 'bowl', 'of', 'food', 'in', 'the', 'middle', '?', '-A-', 'no', '-B-', 'okay', '.', 'how', 'about', 'you', '?']
Second segment decoded dialogue: ['-A-', 'do', 'you', 'have', 'the', 'one', 'with', 'two', 'plates', 'and', 'the', 'salsa', 'in', 'the', 'middle', '?', '-B-', 'i', 'do', "n't", 'have', 'that', 'one']
Reference chain and segments' associated image sets:
['-B-', 'two', 'plates', 'of', 'food', 'with', 'a', 'bowl', 'of', 'food', 'in', 'the', 'middle', '?', '-A-', 'no', '-B-', 'okay', '.', 'how', 'about', 'you', '?']


Game ids

In [5]:
with open('data/test_ids.json') as json_file:
    game_ids = json.load(json_file)
game_ids

['...']

# Models

Load model

In [6]:
from get_predictions import get_predictions
from helpers import *

In [7]:
# Get the datasets for experiment split
# dataset_pred_no_hist, dataset_pred_hist_cp = get_pred_datasets(split='test')

In [8]:

# pickle.dump( dataset_pred_no_hist, open( "dataset_pred_no_hist.p", "wb" ) )
# pickle.dump( dataset_pred_hist_cp, open( "dataset_pred_hist_cp.p", "wb" ) )
dataset_pred_no_hist = pickle.load(open( "dataset_pred_no_hist.p", "rb" ) )
dataset_pred_hist_cp = pickle.load(open( "dataset_pred_hist_cp.p", "rb" ) )

In [9]:
# First index
ind = 24
print('Endocing of this segment is', dataset_pred_no_hist[ind]['segment'])
print('Decoded segment is', vocab.decode(dataset_pred_no_hist[ind]['segment']))
print('Length of the segment is', dataset_pred_no_hist[ind]['length'])
print('Images in the round of this segment is', dataset_pred_no_hist[ind]['image_set'])
print('Target of this segment is', dataset_pred_no_hist[ind]['targets'])
print('Rank (e.g. first time or third time this image was referenced in this game)', dataset_pred_no_hist[ind]['ranks'])
print('Predictions of the model on the segment is', dataset_pred_no_hist[ind]['preds'])
print('Loss of the predictions', dataset_pred_no_hist[ind]['loss'])

print(len(dataset_pred_no_hist))

Endocing of this segment is [5, 38, 59, 415, 23, 249, 18, 690, 10, 4, 17]
Decoded segment is ['-B-', 'two', 'pink', 'bowls', 'of', 'rice', 'and', 'broccoli', '?', '-A-', 'yes']
Length of the segment is 11
Images in the round of this segment is ['132002', '310714', '100022', '492731', '356290', '395097', '144797', '86285']
Target of this segment is [7, 2]
Rank (e.g. first time or third time this image was referenced in this game) [1, 1]
Predictions of the model on the segment is tensor([[[0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]]])
Loss of the predictions tensor(12.6820)
6801


## Sort segments into conditions

The following things have been taken into account when creating the dataframes:
- Only the predictions where there was only one target image given a segment (and thus not multiple target images given a segment)
- Only the predictions where the model predicts one image (and thus not multiple images given a segment)

correct_prediction = 1 when the model predicts correctly, 0 when the model predicts wrong

In [10]:
dataframe = get_pred_dataframe(dataset_pred_no_hist, dataset_pred_hist_cp)
    

 18%|█████████████▊                                                             | 1257/6801 [00:00<00:00, 12325.40it/s]

[4, 12, 22, 7, 21, 11, 294, 16, 9, 945, 10, 5, 13, 8, 12, 31] [4, 12, 22, 7, 21, 11, 294, 16, 9, 945, 10, 5, 13, 8, 12, 31]
No history


 66%|█████████████████████████████████████████████████▏                         | 4464/6801 [00:00<00:00, 21713.70it/s]

History


100%|███████████████████████████████████████████████████████████████████████████| 6801/6801 [00:00<00:00, 20639.52it/s]


Sort the segments in their conditions (whether the history/no history dataset predicted them correctly)

hT_nhF - history True, no-history False (so it was correctly predicted with history, incorrectly without)

Conditions_inds is a dictionary of lists, {'condition': [segment indices for this condition]}

In [11]:
conditions_inds = get_conditions_inds(dataframe)

print("Number of segments in conditions")
counter = 0
for key in conditions_inds:
    print(key, len(conditions_inds[key]))
    counter +=len(conditions_inds[key])
counter

Number of segments in conditions
hT_nhT 1702
hT_nhF 112
hF_nhT 121
hF_nhF 248
only_h 681
only_nh 787
nothing 3150


6801

In [12]:
conditions_inds['hT_nhT']

[11,
 12,
 13,
 14,
 15,
 16,
 17,
 21,
 23,
 26,
 29,
 31,
 35,
 36,
 39,
 40,
 41,
 42,
 45,
 46,
 47,
 48,
 49,
 56,
 59,
 63,
 64,
 70,
 71,
 73,
 74,
 75,
 93,
 94,
 97,
 98,
 101,
 102,
 103,
 104,
 105,
 106,
 108,
 118,
 128,
 132,
 136,
 140,
 141,
 143,
 146,
 150,
 152,
 157,
 162,
 163,
 164,
 169,
 170,
 171,
 172,
 173,
 178,
 179,
 182,
 190,
 193,
 195,
 197,
 198,
 199,
 200,
 201,
 203,
 211,
 220,
 222,
 228,
 231,
 232,
 240,
 242,
 243,
 245,
 248,
 249,
 250,
 251,
 254,
 255,
 256,
 258,
 262,
 263,
 266,
 268,
 269,
 270,
 271,
 277,
 288,
 294,
 299,
 302,
 306,
 307,
 308,
 310,
 311,
 313,
 314,
 315,
 316,
 318,
 320,
 321,
 346,
 347,
 353,
 355,
 357,
 369,
 371,
 372,
 375,
 381,
 384,
 392,
 401,
 407,
 419,
 430,
 438,
 443,
 444,
 450,
 465,
 466,
 469,
 471,
 473,
 474,
 478,
 480,
 485,
 489,
 503,
 505,
 509,
 549,
 563,
 575,
 576,
 587,
 597,
 603,
 614,
 619,
 623,
 625,
 627,
 630,
 632,
 640,
 641,
 647,
 659,
 678,
 681,
 684,
 689,
 696,
 700

In [13]:
print("The number of segments per condition that have a history:")
# For each condition
for condition in conditions_inds:
    counter = 0
#     for seg_id in range(len(dataset_pred_hist_cp)):
    # Iterate over the indices in this condition
    for seg_id in conditions_inds[condition]:
        # For each chain that this segment belongs to
        for chain_i in range(len(dataset_pred_hist_cp[seg_id]['chains'])):
            # Add counter if the segment is not the first round
            if dataset_pred_hist_cp[seg_id]['rounds'][chain_i] > 0:
                counter += 1
                break
    print(condition, counter)

The number of segments per condition that have a history:
hT_nhT 1144
hT_nhF 84
hF_nhT 61
hF_nhF 154
only_h 419
only_nh 548
nothing 2439


In [14]:
results_hist, results_nohist, accs_hist, accs_nohist = get_accuracies(conditions_inds, dataframe)
print("history accuracy")
print(accs_hist)
print("no history accuracy")
print(accs_nohist)


history accuracy
{'hT_nhT': 1.0, 'hT_nhF': 1.0, 'hF_nhT': 0.0, 'hF_nhF': 0.0, 'only_h': 0.7430249632892805, 'only_nh': nan, 'nothing': nan}
no history accuracy
{'hT_nhT': 1.0, 'hT_nhF': 0.0, 'hF_nhT': 1.0, 'hF_nhF': 0.0, 'only_h': nan, 'only_nh': 0.7306226175349428, 'nothing': nan}


C:\Users\linda\Documents\Projects\CDM\photobook_dataset\discriminator\helpers.py:204: RuntimeWarning: invalid value encountered in double_scalars
  accs_nohist[condition] = res.sum()/len(res)
C:\Users\linda\Documents\Projects\CDM\photobook_dataset\discriminator\helpers.py:202: RuntimeWarning: invalid value encountered in double_scalars
  accs_hist[condition] = res.sum()/len(res)


## Combine segments with first in the chain for analyses
Create a dict with the first segment and current segment of a condition


In [15]:
condition_seg_hist = get_condition_seg_hist(conditions_inds, dataset_pred_hist_cp)

In [16]:
seg_id = 11
condition_seg_hist['hT_nhT'][13]

{2: {'first_id': 2,
  'first_seg': [4, 21, 11, 757, 18, 6, 26, 49, 10, 5, 41],
  'current_id': 13,
  'current_seg': [5, 549, 23, 435, 51, 29, 26, 49, 10, 4, 13, 5, 80],
  'round': 1}}

TODO: add analyses here?

## Perturbations

If we make a new perturbation:
    
    * create a new segment and chain file, even if one of them is the same as the test file. Leave vocab file just as it is.
    * Make sure the file format is split+_chains.json and split+segments.json, where split is e.g. 'test_shuffle'. Split can be used to indicate the experiment that is run
    * Also let these files be put into the '/data' folder
    * When shuffeling files based on chain_test_set or segments_test set, make a deep copy first!!
    * If you change the segments in the chains json file, make sure you also update the lengths
    
    * add the experiment and the correspnding segment and chain files in the experiment_names dictionary
  

##### extra info
The training model takes these files as input. 

Change chains if we want to switch up the segments (i.e. from other games or change the order)

Change segments if we want to change things within the utterances

In [17]:
# For later analyses
experiment_names = {"standard":{'segments_file':'data/test_chains.json',
                                'chains_file':'data/test_segments.json',
                                'split':'test'},
                   "shuffle":{'segments_file':'data/test_shuffle_chains.json',
                                'chains_file':'data/test_shuffle_segments.json',
                                'split':'test_shuffle'},
                    
                    "pos_tag_noun":{'segments_file':'data/test_noun_segments.json',
                                'chains_file':'data/test_noun_chains.json',
                                'split':'test_noun'},
                    "pos_tag_verb":{'segments_file':'data/test_verb_segments.json',
                                'chains_file':'data/test_verb_chains.json',
                                'split':'test_verb'},
                    "pos_tag_adj":{'segments_file':'data/test_adj_segments.json',
                                'chains_file':'data/test_adj_chains.json',
                                'split':'test_adj'},
                    "pos_tag_adv":{'segments_file':'data/test_adv_segments.json',
                                'chains_file':'data/test_adv_chains.json',
                                'split':'test_adv'},
                    
                   "games_0":{'segments_file':'data/test_games_0_segments.json',
                                'chains_file':'data/test_games_0_chains.json',
                                'split':'test_games_0'},
                   "games_1":{'segments_file':'data/test_games_1_segments.json',
                                'chains_file':'data/test_games_1_chains.json',
                                'split':'test_games_1'},
                   "games_2":{'segments_file':'data/test_games_2_segments.json',
                                'chains_file':'data/test_games_2_chains.json',
                                'split':'test_games_2'},
                   "games_3":{'segments_file':'data/test_games_3_segments.json',
                                'chains_file':'data/test_games_3_chains.json',
                                'split':'test_games_3'},
                   "games_4":{'segments_file':'data/test_games_4_segments.json',
                                'chains_file':'data/test_games_4_chains.json',
                                'split':'test_games_4'},
                    
                   "imgs_0":{'segments_file':'data/test_imgs_0_segments.json',
                                'chains_file':'data/test_imgs_0_chains.json',
                                'split':'test_imgs_0'},   
                   "imgs_1":{'segments_file':'data/test_imgs_1_segments.json',
                                'chains_file':'data/test_imgs_1_chains.json',
                                'split':'test_imgs_1'},   
                   "imgs_2":{'segments_file':'data/test_imgs_2_segments.json',
                                'chains_file':'data/test_imgs_2_chains.json',
                                'split':'test_imgs_2'},   
                   "imgs_3":{'segments_file':'data/test_imgs_3_segments.json',
                                'chains_file':'data/test_imgs_3_chains.json',
                                'split':'test_imgs_3'},   
                   "imgs_4":{'segments_file':'data/test_imgs_4_segments.json',
                                'chains_file':'data/test_imgs_4_chains.json',
                                'split':'test_imgs_4'},   
                   }

for experiment in experiment_names:
    experiment_names[experiment]['accs_hist'] = []
    experiment_names[experiment]['accs_nohist'] = []
    experiment_names[experiment]['lengths'] = []

In [18]:
experiment_names

{'standard': {'segments_file': 'data/test_chains.json',
  'chains_file': 'data/test_segments.json',
  'split': 'test',
  'accs_hist': [],
  'accs_nohist': [],
  'lengths': []},
 'shuffle': {'segments_file': 'data/test_shuffle_chains.json',
  'chains_file': 'data/test_shuffle_segments.json',
  'split': 'test_shuffle',
  'accs_hist': [],
  'accs_nohist': [],
  'lengths': []},
 'pos_tag_noun': {'segments_file': 'data/test_noun_segments.json',
  'chains_file': 'data/test_noun_chains.json',
  'split': 'test_noun',
  'accs_hist': [],
  'accs_nohist': [],
  'lengths': []},
 'pos_tag_verb': {'segments_file': 'data/test_verb_segments.json',
  'chains_file': 'data/test_verb_chains.json',
  'split': 'test_verb',
  'accs_hist': [],
  'accs_nohist': [],
  'lengths': []},
 'pos_tag_adj': {'segments_file': 'data/test_adj_segments.json',
  'chains_file': 'data/test_adj_chains.json',
  'split': 'test_adj',
  'accs_hist': [],
  'accs_nohist': [],
  'lengths': []},
 'pos_tag_adv': {'segments_file': 'data

### Shuffle experiment
Change order of the segments in a chain

In [19]:
import copy
from perturbations import chain_shuffle

# Get a deepcopy of the chain list
chain_test_shuf_ch = copy.deepcopy(chain_test_set.chains)   

# Shuffle segment order
# Make sure output file ends with chains.json. So now 'test_shuf' is the split argument in following functions
data_chain_shuf, data_seg_shuf = chain_shuffle(chain_test_shuf_ch, chain_file='data/test_shuffle_chains.json', segment_file='data/test_shuffle_segments.json')

In [20]:
pert_sanity_check(test_chains_exp='data/test_shuffle_chains.json', test_segm_exp='data/test_shuffle_segments.json')

Sanity check
original chains
{'game_id': '3R2UR8A0IBELL07JV1VG7ISBNQAOX23WMINLGALC18GY2NUMUBSANWXRHCAO', 'chain_id': 3279, 'target': '341060', 'segments': [0, 7, 8], 'lengths': [35, 15, 33]}
experiment chains
{'game_id': '3R2UR8A0IBELL07JV1VG7ISBNQAOX23WMINLGALC18GY2NUMUBSANWXRHCAO', 'chain_id': 3279, 'target': '341060', 'segments': [7, 0, 8], 'lengths': [15, 35, 33]}

original segments
{'segment': [5, 212, 4, 166, 5, 12, 22, 782, 29, 234, 213, 10, 4, 416, 14, 12, 22, 7, 6, 49, 11, 6, 155, 295, 11, 6, 811, 16, 9, 95, 10, 5, 17, 8, 12], 'image_set': ['93469', '380128', '14238', '259745', '524866', '341060', '483794', '96757'], 'targets': [5], 'length': 35}
experiment segments
{'segment': [5, 212, 4, 166, 5, 12, 22, 782, 29, 234, 213, 10, 4, 416, 14, 12, 22, 7, 6, 49, 11, 6, 155, 295, 11, 6, 811, 16, 9, 95, 10, 5, 17, 8, 12], 'image_set': ['93469', '380128', '14238', '259745', '524866', '341060', '483794', '96757'], 'targets': [5], 'length': 35}
-----------------------------------
origin

## Game exchange experiment
For a certain round round_id, change the segment in that round for a segment on the same image from the same round but a random game

In [21]:
# def check_lengths(changed_seg_ids, conditions_inds):
#     """Return a dictionary of the number of segments that were changed per condition"""
#     lengths = {'hT_nhT':0, 'hT_nhF':0,'hF_nhT':0,'hF_nhF':0,'only_h':0,'only_nh':0,'nothing':0}
#     for seg_id in changed_seg_ids:
#         for condition in conditions_inds:
#             if seg_id in conditions_inds[condition]:
#                 lengths[condition]+=1
#                 break
#     return lengths
    
    
# # def run_exp_games(round_id, chains, split):
# #     """
# #     chains = chain_test_set.chains
# #     """
# #     # Create new segment and chain file
# # #     split = 'test_games_'+str(round_id)
# #     chain_copy = copy.deepcopy(chains)   
# # #     chain_test_game_n, segments, changed_seg_ids = pert_exchange_games(chain_copy, round_id, chain_file_start='data/test_games_', segment_file_start='data/test_games_', output=True)
# # #     pickle.dump( changed_seg_ids, open( "data/"+split+ "_changed_seg_ids.p", "wb" ) )
# #     changed_seg_ids = pickle.load(open( 'data/test_games_1_changed_seg_ids.p', "rb" ) )
    
# #     # Get dataframes
# #     # TODO: remove pickles
# # #     dataset_pred_no_hist_exp, dataset_pred_hist_cp_exp = get_pred_datasets(split=split)
# #     dataset_pred_no_hist_exp = pickle.load(open( "dataset_pred_no_hist_exp.p", "rb" ) )
# #     dataset_pred_hist_cp_exp = pickle.load(open( "dataset_pred_hist_cp_exp.p", "rb" ) )
    

# #     # Get predictions of the segments
# #     dataframe_exp = get_pred_dataframe(dataset_pred_no_hist_exp, dataset_pred_hist_cp_exp)
    
# #     # Get accuracies
# #     results_hist, results_nohist, accs_hist, accs_nohist = get_accuracies_seg(conditions_inds, dataframe_exp, changed_seg_ids)
# #     # Get the number of segments that were changed per condition
# #     lengths = check_lengths(changed_seg_ids, conditions_inds)
    
# #     return accs_hist, accs_nohist, results_hist, results_nohist, lengths
    

In [22]:
chain_copy = copy.deepcopy(chain_test_set.chains)
chain_test_img_n, segments, changed_seg_ids = pert_exchange_games(chain_copy, 1, chain_file_start='data/test_games_', segment_file_start='data/test_games_', output=True)


In [23]:
pert_sanity_check(test_chains_exp='data/test_games_1_chains.json', test_segm_exp='data/test_games_1_segments.json')

Sanity check
original chains
{'game_id': '3R2UR8A0IBELL07JV1VG7ISBNQAOX23WMINLGALC18GY2NUMUBSANWXRHCAO', 'chain_id': 3279, 'target': '341060', 'segments': [0, 7, 8], 'lengths': [35, 15, 33]}
experiment chains
{'game_id': '3R2UR8A0IBELL07JV1VG7ISBNQAOX23WMINLGALC18GY2NUMUBSANWXRHCAO', 'chain_id': 3279, 'target': '341060', 'segments': [0, 6250, 8], 'lengths': [35, 20, 33]}

original segments
{'segment': [5, 212, 4, 166, 5, 12, 22, 782, 29, 234, 213, 10, 4, 416, 14, 12, 22, 7, 6, 49, 11, 6, 155, 295, 11, 6, 811, 16, 9, 95, 10, 5, 17, 8, 12], 'image_set': ['93469', '380128', '14238', '259745', '524866', '341060', '483794', '96757'], 'targets': [5], 'length': 35}
experiment segments
{'segment': [5, 212, 4, 166, 5, 12, 22, 782, 29, 234, 213, 10, 4, 416, 14, 12, 22, 7, 6, 49, 11, 6, 155, 295, 11, 6, 811, 16, 9, 95, 10, 5, 17, 8, 12], 'image_set': ['93469', '380128', '14238', '259745', '524866', '341060', '483794', '96757'], 'targets': [5], 'length': 35}
-----------------------------------
ori

## Image exchange experiment
For a round_id, change the segment for another segment on another target image but from the same game

In [24]:
for round_id in [0, 1, 2, 3, 4]:
    chain_copy = copy.deepcopy(chain_test_set.chains)   
    chain_test_game_n, segments, changed_seg_ids = pert_exchange_imgs(chain_copy, round_id, chain_file_start='data/test_imgs_', segment_file_start='data/test_imgs_', output=True)


In [25]:
pert_sanity_check(test_chains_exp='data/test_imgs_1_chains.json', test_segm_exp='data/test_imgs_1_segments.json')

Sanity check
original chains
{'game_id': '3R2UR8A0IBELL07JV1VG7ISBNQAOX23WMINLGALC18GY2NUMUBSANWXRHCAO', 'chain_id': 3279, 'target': '341060', 'segments': [0, 7, 8], 'lengths': [35, 15, 33]}
experiment chains
{'game_id': '3R2UR8A0IBELL07JV1VG7ISBNQAOX23WMINLGALC18GY2NUMUBSANWXRHCAO', 'chain_id': 3279, 'target': '341060', 'segments': [0, 13, 8], 'lengths': [35, 13, 33]}

original segments
{'segment': [5, 212, 4, 166, 5, 12, 22, 782, 29, 234, 213, 10, 4, 416, 14, 12, 22, 7, 6, 49, 11, 6, 155, 295, 11, 6, 811, 16, 9, 95, 10, 5, 17, 8, 12], 'image_set': ['93469', '380128', '14238', '259745', '524866', '341060', '483794', '96757'], 'targets': [5], 'length': 35}
experiment segments
{'segment': [5, 212, 4, 166, 5, 12, 22, 782, 29, 234, 213, 10, 4, 416, 14, 12, 22, 7, 6, 49, 11, 6, 155, 295, 11, 6, 811, 16, 9, 95, 10, 5, 17, 8, 12], 'image_set': ['93469', '380128', '14238', '259745', '524866', '341060', '483794', '96757'], 'targets': [5], 'length': 35}
-----------------------------------
origi

## Run perturbation predictions
Get the accuracies per condition for an experiment.
The experiments will be saved in the experiment_names dictionary.

In [27]:
def experiment_accs(conditions_inds, split='test'):
    # Get dataframes
    dataset_pred_no_hist_exp, dataset_pred_hist_cp_exp = get_pred_datasets(split=split)
    # Get predictions of the segments
    dataframe_exp = get_pred_dataframe(dataset_pred_no_hist_exp, dataset_pred_hist_cp_exp)
    # Get the accuracy dict
    results_hist, results_nohist, accs_hist, accs_nohist = get_accuracies(conditions_inds, dataframe_exp)
    
    return accs_hist, accs_nohist, dataframe_exp, dataset_pred_hist_cp_exp, dataset_pred_no_hist_exp
    
def run_experiment(experiment):

    split = experiment_names[experiment]['split']
    accs_hist, accs_nohist, dataframe_exp, dataset_pred_hist_cp_exp, dataset_pred_no_hist_exp = experiment_accs(conditions_inds, split=split)

    print(f"Experiment{experiment} accuracies:")
    print("history:")
    print(accs_hist)
    print("No history")
    print(accs_nohist)

    return accs_hist, accs_nohist

def run_exp_games_imgs(round_id, chains, split, exp='games'):
    """
    chains = chain_test_set.chains
    """
    # Create new segment and chain file
    chain_copy = copy.deepcopy(chains)
    if exp == 'games':
        chain_test_game_n, segments, changed_seg_ids = pert_exchange_games(chain_copy, round_id, chain_file_start='data/test_games_', segment_file_start='data/test_games_', output=True)
    elif exp == 'imgs':
        chain_test_img_n, segments, changed_seg_ids = pert_exchange_imgs(chain_copy, round_id, chain_file_start='data/test_imgs_', segment_file_start='data/test_imgs_', output=True)

    # Get dataframes
    dataset_pred_no_hist_exp, dataset_pred_hist_cp_exp = get_pred_datasets(split=split)

    # Get predictions of the segments
    dataframe_exp = get_pred_dataframe(dataset_pred_no_hist_exp, dataset_pred_hist_cp_exp)
    
    # Get accuracies
    results_hist, results_nohist, accs_hist, accs_nohist = get_accuracies_seg(conditions_inds, dataframe_exp, changed_seg_ids)
    # Get the number of segments that were changed per condition
    lengths = check_lengths(changed_seg_ids, conditions_inds)
    
    return accs_hist, accs_nohist, results_hist, results_nohist, lengths


In [28]:

def experiment_results(experiment_names, chains, experiment='standard', prob=1):
    """
    Run an experiment [standard, shuffle, pos_tag_noun..., games_0..., imgs_0...]
    round_id is the round for the games and imgs experiments
    prop is the proportion of the pos tags that are to be removed
    n is number of times 
    """
    split = experiment_names[experiment]['split']
    
    if experiment == "standard":
        accs_hist, accs_nohist = run_experiment(experiment)
        lengths = {condition: len(conditions_inds[condition])for condition in conditions_inds}
    
    # TODO: ADD CREATION OF FILES HERE
    elif experiment == "shuffle":
        accs_hist, accs_nohist = run_experiment(experiment)
        lengths = {condition: len(conditions_inds[condition])for condition in conditions_inds}

    # POS experiments
    # TODO: ADD CREATION OF FILES HERE
    # TODO: ADD PERCENTAGE HERE
    elif experiment == "pos_tag_noun":
        accs_hist, accs_nohist = run_experiment(experiment)
        lengths = {condition: len(conditions_inds[condition])for condition in conditions_inds}

    elif experiment == "pos_tag_verb":
        accs_hist, accs_nohist = run_experiment(experiment)
        lengths = {condition: len(conditions_inds[condition])for condition in conditions_inds}

    elif experiment == "pos_tag_adj":
        accs_hist, accs_nohist = run_experiment(experiment)
        lengths = {condition: len(conditions_inds[condition])for condition in conditions_inds}

    elif experiment == "pos_tag_adv":
        accs_hist, accs_nohist = run_experiment(experiment)
        lengths = {condition: len(conditions_inds[condition])for condition in conditions_inds}

    # Game exchange
    elif experiment.startswith('games'):
        round_id = int(experiment[-1])
        accs_hist, accs_nohist, results_hist, results_nohist, lengths = run_exp_games_imgs(round_id, chains, split, exp='games')
    
    # Imgs exchange
    elif experiment.startswith('imgs'):
        round_id = int(experiment[-1])
        accs_hist, accs_nohist, results_hist, results_nohist, lengths = run_exp_games_imgs(round_id, chains, split, exp='imgs')
    
    
    # Save the accuracies to the experiment_names dict
    experiment_names[experiment]['accs_hist'].append(accs_hist)
    experiment_names[experiment]['accs_nohist'].append(accs_nohist)
    experiment_names[experiment]['lengths'].append(lengths)
    return experiment_names, accs_hist, accs_nohist, lengths

### Test if it works for games_1, imgs_1, and shuffle

In [33]:
# experiment_names, accs_hist, accs_nohist, lengths = experiment_results(experiment_names, chains=chain_test_set.chains,
#                                                                        experiment='games_1', prob=1)
# experiment_names

loaded model: No history
Initialising vocab from file.
vocab len 3424
Initialising vocab from file.
Loaded seg2ranks and idlist
params. normalize=True, mask=True, weight=5.5, weighting=True, batchsize=1, breaking=False
Dataparams. data_dir=./data, segmentfile=segments.json, vectorfile=vectors.json, chains_file=chains.json
segment dataset with segments.json vectors.json test_games_1
history dataset with test_games_1_segments.json vectors.json test_games_1_chains.json test_games_1
processing test_games_1

Start prediction
predict no history
{'segment': [5, 212, 4, 166, 5, 12, 22, 782, 29, 234, 213, 10, 4, 416, 14, 12, 22, 7, 6, 49, 11, 6, 155, 295, 11, 6, 811, 16, 9, 95, 10, 5, 17, 8, 12], 'image_set': ['93469', '380128', '14238', '259745', '524866', '341060', '483794', '96757'], 'targets': [5], 'length': 35, 'preds': tensor([[[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]]]), 'loss': tensor(4.9204), 'ranks': [0]}
getting pre

 25%|███████████████████                                                        | 1726/6801 [00:00<00:00, 17134.92it/s]

[4, 12, 22, 7, 21, 11, 294, 16, 9, 945, 10, 5, 13, 8, 12, 31] [4, 12, 22, 7, 21, 11, 294, 16, 9, 945, 10, 5, 13, 8, 12, 31]
No history


 40%|██████████████████████████████▏                                            | 2733/6801 [00:00<00:00, 27132.21it/s]

History


100%|███████████████████████████████████████████████████████████████████████████| 6801/6801 [00:00<00:00, 24180.70it/s]


{'standard': {'segments_file': 'data/test_chains.json',
  'chains_file': 'data/test_segments.json',
  'split': 'test',
  'accs_hist': [],
  'accs_nohist': [],
  'lengths': []},
 'shuffle': {'segments_file': 'data/test_shuffle_chains.json',
  'chains_file': 'data/test_shuffle_segments.json',
  'split': 'test_shuffle',
  'accs_hist': [{'hT_nhT': 0.961082910321489,
    'hT_nhF': 0.8227848101265823,
    'hF_nhT': 0.2631578947368421,
    'hF_nhF': 0.23493975903614459,
    'only_h': 0.7780269058295964,
    'only_nh': 0.7419354838709677,
    'nothing': 0.793400286944046}],
  'accs_nohist': [{'hT_nhT': 1.0,
    'hT_nhF': 0.0,
    'hF_nhT': 1.0,
    'hF_nhF': 0.0,
    'only_h': nan,
    'only_nh': 0.7306226175349428,
    'nothing': nan}],
  'lengths': [{'hT_nhT': 1702,
    'hT_nhF': 112,
    'hF_nhT': 121,
    'hF_nhF': 248,
    'only_h': 681,
    'only_nh': 787,
    'nothing': 3150}]},
 'pos_tag_noun': {'segments_file': 'data/test_noun_segments.json',
  'chains_file': 'data/test_noun_chains.js

In [30]:
# experiment_names, accs_hist, accs_nohist, lengths = experiment_results(experiment_names, chains=chain_test_set.chains,
#                                                                        experiment='imgs_1', prob=1)
# experiment_names

loaded model: No history
Initialising vocab from file.
vocab len 3424
Initialising vocab from file.
Loaded seg2ranks and idlist
params. normalize=True, mask=True, weight=5.5, weighting=True, batchsize=1, breaking=False
Dataparams. data_dir=./data, segmentfile=segments.json, vectorfile=vectors.json, chains_file=chains.json
segment dataset with segments.json vectors.json test_imgs_1
history dataset with test_imgs_1_segments.json vectors.json test_imgs_1_chains.json test_imgs_1
processing test_imgs_1

Start prediction
predict no history
{'segment': [5, 212, 4, 166, 5, 12, 22, 782, 29, 234, 213, 10, 4, 416, 14, 12, 22, 7, 6, 49, 11, 6, 155, 295, 11, 6, 811, 16, 9, 95, 10, 5, 17, 8, 12], 'image_set': ['93469', '380128', '14238', '259745', '524866', '341060', '483794', '96757'], 'targets': [5], 'length': 35, 'preds': tensor([[[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]]]), 'loss': tensor(4.9204), 'ranks': [0]}
getting predicit

 32%|███████████████████████▉                                                   | 2174/6801 [00:00<00:00, 21579.24it/s]

[4, 12, 22, 7, 21, 11, 294, 16, 9, 945, 10, 5, 13, 8, 12, 31] [4, 12, 22, 7, 21, 11, 294, 16, 9, 945, 10, 5, 13, 8, 12, 31]
No history


 37%|███████████████████████████▉                                               | 2529/6801 [00:00<00:00, 25106.91it/s]

History


100%|███████████████████████████████████████████████████████████████████████████| 6801/6801 [00:00<00:00, 24267.67it/s]


{'standard': {'segments_file': 'data/test_chains.json',
  'chains_file': 'data/test_segments.json',
  'split': 'test',
  'accs_hist': [],
  'accs_nohist': [],
  'lengths': []},
 'shuffle': {'segments_file': 'data/test_shuffle_chains.json',
  'chains_file': 'data/test_shuffle_segments.json',
  'split': 'test_shuffle',
  'accs_hist': [],
  'accs_nohist': [],
  'lengths': []},
 'pos_tag_noun': {'segments_file': 'data/test_noun_segments.json',
  'chains_file': 'data/test_noun_chains.json',
  'split': 'test_noun',
  'accs_hist': [],
  'accs_nohist': [],
  'lengths': []},
 'pos_tag_verb': {'segments_file': 'data/test_verb_segments.json',
  'chains_file': 'data/test_verb_chains.json',
  'split': 'test_verb',
  'accs_hist': [],
  'accs_nohist': [],
  'lengths': []},
 'pos_tag_adj': {'segments_file': 'data/test_adj_segments.json',
  'chains_file': 'data/test_adj_chains.json',
  'split': 'test_adj',
  'accs_hist': [],
  'accs_nohist': [],
  'lengths': []},
 'pos_tag_adv': {'segments_file': 'data

In [31]:
# experiment_names, accs_hist, accs_nohist, lengths = experiment_results(experiment_names, chains=chain_test_set.chains,
#                                                                        experiment='shuffle', prob=1)
# experiment_names

loaded model: No history
Initialising vocab from file.
vocab len 3424
Initialising vocab from file.
Loaded seg2ranks and idlist
params. normalize=True, mask=True, weight=5.5, weighting=True, batchsize=1, breaking=False
Dataparams. data_dir=./data, segmentfile=segments.json, vectorfile=vectors.json, chains_file=chains.json
segment dataset with segments.json vectors.json test_shuffle
history dataset with test_shuffle_segments.json vectors.json test_shuffle_chains.json test_shuffle
processing test_shuffle

Start prediction
predict no history
{'segment': [5, 212, 4, 166, 5, 12, 22, 782, 29, 234, 213, 10, 4, 416, 14, 12, 22, 7, 6, 49, 11, 6, 155, 295, 11, 6, 811, 16, 9, 95, 10, 5, 17, 8, 12], 'image_set': ['93469', '380128', '14238', '259745', '524866', '341060', '483794', '96757'], 'targets': [5], 'length': 35, 'preds': tensor([[[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]]]), 'loss': tensor(4.9204), 'ranks': [1]}
getting pre

 34%|█████████████████████████▎                                                 | 2292/6801 [00:00<00:00, 22755.26it/s]

[5, 294, 51, 29, 49, 10, 4, 13, 5, 80] [4, 12, 22, 7, 21, 11, 294, 16, 9, 945, 10, 5, 13, 8, 12, 31]
No history


 70%|████████████████████████████████████████████████████▍                      | 4760/6801 [00:00<00:00, 23441.47it/s]

History


100%|███████████████████████████████████████████████████████████████████████████| 6801/6801 [00:00<00:00, 23476.97it/s]

Experimentshuffle accuracies:
history:
{'hT_nhT': 0.961082910321489, 'hT_nhF': 0.8227848101265823, 'hF_nhT': 0.2631578947368421, 'hF_nhF': 0.23493975903614459, 'only_h': 0.7780269058295964, 'only_nh': 0.7419354838709677, 'nothing': 0.793400286944046}
No history
{'hT_nhT': 1.0, 'hT_nhF': 0.0, 'hF_nhT': 1.0, 'hF_nhF': 0.0, 'only_h': nan, 'only_nh': 0.7306226175349428, 'nothing': nan}


{'standard': {'segments_file': 'data/test_chains.json',
  'chains_file': 'data/test_segments.json',
  'split': 'test',
  'accs_hist': [],
  'accs_nohist': [],
  'lengths': []},
 'shuffle': {'segments_file': 'data/test_shuffle_chains.json',
  'chains_file': 'data/test_shuffle_segments.json',
  'split': 'test_shuffle',
  'accs_hist': [{'hT_nhT': 0.961082910321489,
    'hT_nhF': 0.8227848101265823,
    'hF_nhT': 0.2631578947368421,
    'hF_nhF': 0.23493975903614459,
    'only_h': 0.7780269058295964,
    'only_nh': 0.7419354838709677,
    'nothing': 0.793400286944046}],
  'accs_nohist': [{'hT_nhT': 1.0,
    'hT_nhF': 0.0,
    'hF_nhT': 1.0,
    'hF_nhF': 0.0,
    'only_h': nan,
    'only_nh': 0.7306226175349428,
    'nothing': nan}],
  'lengths': [{'hT_nhT': 1702,
    'hT_nhF': 112,
    'hF_nhT': 121,
    'hF_nhF': 248,
    'only_h': 681,
    'only_nh': 787,
    'nothing': 3150}]},
 'pos_tag_noun': {'segments_file': 'data/test_noun_segments.json',
  'chains_file': 'data/test_noun_chains.js

### Run experiments multiple times

Game experiments

In [92]:
n_exp = 3
for i in range(n_exp):
    for round_id in range(5):
        experiment_names, accs_hist, accs_nohist, lengths = experiment_results(experiment_names, chains=chain_test_set.chains,
                                                                               experiment='games_'+str(round_id), prob=1)
experiment_names

 34%|█████████████████████████▌                                                 | 2321/6801 [00:00<00:00, 22983.98it/s]

[4, 12, 22, 7, 21, 11, 294, 16, 9, 945, 10, 5, 13, 8, 12, 31] [4, 12, 22, 7, 21, 11, 294, 16, 9, 945, 10, 5, 13, 8, 12, 31]
No history


 32%|███████████████████████▉                                                   | 2176/6801 [00:00<00:00, 21551.71it/s]

History


100%|███████████████████████████████████████████████████████████████████████████| 6801/6801 [00:00<00:00, 19221.63it/s]
C:\Users\linda\Miniconda3\envs\NLP_probe\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars
 34%|█████████████████████████▏                                                 | 2286/6801 [00:00<00:00, 22637.66it/s]

...

[4, 12, 22, 7, 21, 11, 294, 16, 9, 945, 10, 5, 13, 8, 12, 31] [4, 12, 22, 7, 21, 11, 294, 16, 9, 945, 10, 5, 13, 8, 12, 31]
No history


 44%|████████████████████████████████▉                                          | 2990/6801 [00:00<00:00, 15214.64it/s]

History


100%|███████████████████████████████████████████████████████████████████████████| 6801/6801 [00:00<00:00, 12321.42it/s]


{'standard': {'segments_file': 'data/test_chains.json',
  'chains_file': 'data/test_segments.json',
  'split': 'test',
  'accs_hist': [],
  'accs_nohist': [],
  'lengths': []},
 'shuffle': {'segments_file': 'data/test_shuffle_chains.json',
  'chains_file': 'data/test_shuffle_segments.json',
  'split': 'test_shuffle',
  'accs_hist': [],
  'accs_nohist': [],
  'lengths': []},
 'pos_tag_noun': {'segments_file': 'data/test_noun_segments.json',
  'chains_file': 'data/test_noun_chains.json',
  'split': 'test_noun',
  'accs_hist': [],
  'accs_nohist': [],
  'lengths': []},
 'pos_tag_verb': {'segments_file': 'data/test_verb_segments.json',
  'chains_file': 'data/test_verb_chains.json',
  'split': 'test_verb',
  'accs_hist': [],
  'accs_nohist': [],
  'lengths': []},
 'pos_tag_adj': {'segments_file': 'data/test_adj_segments.json',
  'chains_file': 'data/test_adj_chains.json',
  'split': 'test_adj',
  'accs_hist': [],
  'accs_nohist': [],
  'lengths': []},
 'pos_tag_adv': {'segments_file': 'data

In [32]:
experiment_names

{'standard': {'segments_file': 'data/test_chains.json',
  'chains_file': 'data/test_segments.json',
  'split': 'test',
  'accs_hist': [],
  'accs_nohist': [],
  'lengths': []},
 'shuffle': {'segments_file': 'data/test_shuffle_chains.json',
  'chains_file': 'data/test_shuffle_segments.json',
  'split': 'test_shuffle',
  'accs_hist': [{'hT_nhT': 0.961082910321489,
    'hT_nhF': 0.8227848101265823,
    'hF_nhT': 0.2631578947368421,
    'hF_nhF': 0.23493975903614459,
    'only_h': 0.7780269058295964,
    'only_nh': 0.7419354838709677,
    'nothing': 0.793400286944046}],
  'accs_nohist': [{'hT_nhT': 1.0,
    'hT_nhF': 0.0,
    'hF_nhT': 1.0,
    'hF_nhF': 0.0,
    'only_h': nan,
    'only_nh': 0.7306226175349428,
    'nothing': nan}],
  'lengths': [{'hT_nhT': 1702,
    'hT_nhF': 112,
    'hF_nhT': 121,
    'hF_nhF': 248,
    'only_h': 681,
    'only_nh': 787,
    'nothing': 3150}]},
 'pos_tag_noun': {'segments_file': 'data/test_noun_segments.json',
  'chains_file': 'data/test_noun_chains.js

In [ ]:
pickle.dump( experiment_names, open( "experiment_names_game_2405.p", "wb" ) )


In [ ]:
n_exp = 3
for i in range(n_exp):
    for round_id in range(5):
        experiment_names, accs_hist, accs_nohist, lengths = experiment_results(experiment_names, chains=chain_test_set.chains,
                                                                               experiment='imgs_'+str(round_id), prob=1)
experiment_names

In [ ]:
pickle.dump( experiment_names, open( "experiment_names_game_img_2405.p", "wb" ) )
